In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from PIL import Image

import torch
from RealESRGAN import RealESRGAN
import pytesseract
from pytesseract import Output

c:\Users\dangc\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\dangc\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\amp\autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


In [17]:
img = cv2.imread('sample_data/box_toy.jpg')
# img = np.rot90(img)
# plt.imshow(img)

# Use Tesseract

- Page segmentation modes available in Tesseract:

0: Orientation and script detection (Orientation and Script Detection-OSD) only.

1: Automatic page segmentation with OSD.

2: Automatic page segmentation, but no OSD, or OCR (optical character recognition).

3: Fully automatic page segmentation, but no OSD. (Default)

4: Assume a single column of text of variable sizes.

5: Assume a single uniform block of vertically aligned text.

6: Assume a single uniform block of text.

7: Treat the image as a single text line.

8: Treat the image as a single word.

9: Treat the image as a single word in a circle.

10: Treat the image as a single character.

11: Sparse text. Find as much text as possible in no particular order.

12: Sparse text with OSD.

13: Raw line. Treat the image as a single text line, bypassing hacks that are Tesseract-specific.

- OCR Engine Modes:

OEM 0: Legacy engine only

OEM 1: Neural nets LSTM engine only

OEM 2: Legacy + LSTM engines

OEM 3: Default, based on what is available

Download pretrained weights for RealESRGAN there: https://drive.google.com/drive/folders/16PlVKhTNkSyWFx52RPb2hXPIQveNGbxS

In [3]:
model = RealESRGAN('cpu', scale=4)
model.load_weights('weights/RealESRGAN_x4.pth')

In [184]:
# sr_image = model.predict(img)
# plt.imshow(sr_image)

### xám hóa ảnh

In [4]:
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray_img.shape

(1093, 832)

In [7]:
# pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
# path = 'C:\Program Files\Tesseract-OCR\tessdata'
config = rf' --psm 11 --oem 3'
img = cv2.imread('sample_data/box_toy.jpg')
print(pytesseract.image_to_string(img, config=config, output_type=Output.DICT))
data = pytesseract.image_to_data(img, config=config, output_type=Output.DICT)

n_boxes = len(data['text'])
for i in range(n_boxes):
    if float(data['conf'][i]) > 0:
        x, y, w, h = int(data['left'][i]), int(data['top'][i]), int(data['width'][i]), int(data['height'][i])
        img_r = cv2.rectangle(img, (x,y), (x + w, y + h), (0, 255, 0), 2)
        img_r = cv2.putText(img_r, data['text'][i], (x, y+h+20), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.7, (255,0,0), 2)

output_image = Image.fromarray(img_r)
output_image.show()

{'text': 'ae OE\n\ny =\n\n—\n\na ®\n\n=>\n\n9/10\n\nA\n\n-\n\n5)\n\nASTON MARTIN V12 SPEEDSTER\n\nto\n'}


# Use TrOCR model

In [8]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel

processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-printed")
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-printed")

c:\Users\dangc\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\dangc\.cache\huggingface\hub\models--microsoft--trocr-base-printed. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
c:\Users\dangc\AppData\Local\Programs\Python\Python311\Lib\site-packages\transfor

In [33]:
img1 = Image.open("sample_data/box_toy.jpg")
img1.show()

In [34]:
pixel_values = processor(images=img1, return_tensors="pt").pixel_values

In [35]:
# Generate text from the image
generated_ids = model.generate(pixel_values)
predicted_text = processor.batch_decode(generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)

# Print the predicted text
print("Predicted Text:", predicted_text)


Predicted Text: ['PAX']
